# s-crvUSD deployment
This script deploys the scrvUSD vault contract to the Ethereum network. Additionally, RewardsHandler contract periphery is deployed to integrate the scrvUSD into the flow of crvUSD lending fees.

## Setup

In [ ]:
import os

# importing scripting dependencies
import boa
from eth_account import Account
from boa import NetworkEnv, Env

In [ ]:
# use address book for relevant addresses
import sys

if os.getcwd()[-7:] == "scripts":
    sys.path.append("..")  # to enable import from parent directory
import tests.integration.address_book as ab

from dotenv import load_dotenv

load_dotenv()

# get env vars
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
PINATA_API_KEY = os.getenv("PINATA_API_KEY")
RPC_URL = os.environ.get("ETH_RPC_URL")
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")

## Select mode
Production mode cell is intentionally executed first to avoid deploying things by accident. If one runs all the cells sequentially things will be executed in forking mode.

To deploy in production mode, manually skip the execution of the fork mode cell.

In [ ]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = NetworkEnv(RPC_URL)
boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)

In [ ]:
# fork mode (for testing)
# we impersonate convex since they definitely have enough to push a vote
CONVEX_VOTERPROXY = "0x989aeb4d175e16225e39e87d0d97a3360524ad80"

forked_env = Env()
boa.set_env(forked_env)

boa.env.fork(RPC_URL, cache_file=None)  # no cache because kernel locks access to the file
boa.env.eoa = CONVEX_VOTERPROXY

# I. Deployment

### 1. Vault

In [ ]:
# I. First deploy the Vault
vault_factory = boa.from_etherscan(
    ab.yearn_vault_factory, "vault_factory", api_key=ETHERSCAN_API_KEY
)

deployer_address = boa.env.eoa
vault_address = vault_factory.deploy_new_vault(
    ab.crvusd,  # underlying token
    "Savings crvUSD",  # vault name
    "scrvUSD",  # vault symbol
    deployer_address,  # initial role manager = deployer
    86_400 * 7,  # unlock time for rewards
)

print(f"Vault deployed at {vault_address}")

### 2. DepositLimitModule

In [ ]:
# II. Then deploy the DepositLimitModule
DepositLimit_deployer = boa.load_partial("../contracts/DepositLimitModule.vy")

deposit_limit = DepositLimit_deployer(
    vault_address,  # vault
    5_000_000 * 10**18,  # cap deposits to 5M crvUSD
    ab.dev_multisig,  # admin
)

print(f"Deposit limit module deployed at {deposit_limit.address}")

### 3. Stablecoin Lens

In [ ]:
# III. Now deploy Stablecoin Lens
StablecoinLens_deployer = boa.load_partial("../contracts/StablecoinLens.vy")

stablecoin_lens = StablecoinLens_deployer(ab.crvusd_controller_factory)

print(f"Stablecoin lens deployed at {stablecoin_lens.address}")

### 4. RewardsHandler

In [ ]:
# IV. Finally deploy the RewardsHandler
RewardsHandler_deployer = boa.load_partial("../contracts/RewardsHandler.vy")

rewards_handler = RewardsHandler_deployer(
    ab.crvusd,  # stablecoin
    vault_address,  # vault
    stablecoin_lens,  # lens
    500,  # minimum weight (5%)
    10_000,  # scaling factor (over MAX_BPS)
    ab.dao_agent,  # WE CERTAIN ABOUT THIS CONTRACT? [TODO]
)

print(f"Rewards handler deployed at {rewards_handler.address}")

# II. Post-deployment setup

In [ ]:
# I. Set the vault
vault = boa.load_partial("../contracts/yearn/VaultV3.vy").at(vault_address)
# a. set the rewards handler roles in the vault
vault.set_role(
    rewards_handler.address, 2**11 | 2**5
)  # set RH as REPORTING_MANAGER and PROFIT_UNLOCK_MANAGER
# b. set the deposit limit module in the vault
vault.set_role(ab.dev_multisig, int("11111111111111", 2))  # set dev_multisig megarole
vault.transfer_role_manager(ab.dao_agent)

# with boa.env.prank(ab.dev_multisig): ### TODO call from dev_multisig
#     vault.set_deposit_limit_module(deposit_limit.address, True)
# c. relinquish the role manager role to DAO ### TODO propose acceptance of role by dao

# II. Set the deposit limit module
# use dev_multisig as admin at deploy time, then manually set the security agent to fuzzland (to avoid extra txns from deployer)
# deposit_limit_address.set_security_agent(ab.fuzzland_address, True) # TODO dev_multisig txn

# III. Finally set the rewards handler parameters
# rewards_handler_contract = boa.load_partial("../contracts/RewardsHandler.vy").at(rewards_handler_address)
# here distribution_time must be set to match the vault's value, but why does RH have distribution_time at all? TODO: discuss

## 3. Vote in the DAO

In [ ]:
import curve_dao

change_fee_splitter_receivers = (
    ab.fee_splitter,
    "set_receivers",
    (
        [
            (vault_address, 1_000),
            (ab.crvusd_fee_collector, 9_000),
        ]
    ),
)

accept_vault_role_manager = (vault, "accept_role_manager", ())

ACTIONS = [change_fee_splitter_receivers, accept_vault_role_manager]
DESCRIPTION = "This proposal starts streaming a part of crvUSD fees to scrvUSD. To enact this the fee splitter will now send (at least) 90% of the revenues to the fee collector (previously 100%), and (at most) 10%, which might be reached only if the number of depositors grows, to sustain the yield."
curve_dao.create_vote(
    curve_dao.get_dao_parameters("ownership"),
    ACTIONS,
    DESCRIPTION,
    ETHERSCAN_API_KEY,
    PINATA_API_KEY,
)  # ACTIONS = [(ab.controller_factory, "set_fee_receiver", fs)]